In [1]:
import pandas as pd
import os
from datetime import datetime


In [2]:
ndmi_dir = 'data/NDMI_Weekly_Exports/NDMI_Weekly_Exports_2019_2024'

In [4]:
df = pd.read_csv('openweathermap_data.csv')
df['precipitation_value'] = df['precipitation'].str.extract(r"'total': ([\d.]+)").astype(float)

In [5]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def calculate_precipitation_sums(df, tiff_folder):
    # Get list of TIFF files
    tiff_files = [f for f in os.listdir(tiff_folder) if f.endswith('.tif')]
    
    results = []
    
    for tiff_file in tiff_files:
        # Extract dates from filename
        dates = tiff_file.replace('NDMI_', '').replace('.tif', '').split('_to_')
        start_date = datetime.strptime(dates[0], '%Y-%m-%d')
        end_date = datetime.strptime(dates[1], '%Y-%m-%d')
        
        # Get precipitation sum for date range
        mask = (df.date >= start_date) & (df.date <= end_date)
        precip_sum = df.loc[mask, 'precipitation_value'].sum()
        
        results.append({
            'tiff_file': tiff_file,
            'start_date': start_date,
            'end_date': end_date,
            'precipitation_sum': precip_sum
        })
    
    # Create new dataframe with results
    result_df = pd.DataFrame(results)
    return result_df

In [9]:
result_df = calculate_precipitation_sums(df, ndmi_dir)
result_df.to_csv('precipitation_sums.csv', index=False)